In [8]:
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import dataframe_image as dfi



def create_ranking_table(t,file_path):
    models = ['Linear','KNN','G-Boost','MLP']
    names = ['Linear*','KNN*','G-Boost*','MLP*']
    rankings = None
    mae_baseline = []
    for i,model in enumerate(models):
        file = "../data/ranking_NPI/" + model + t +".csv"
        df = pd.read_csv(file,index_col=0)
        mae_baseline.append(df['mae_baseline'].values[0])
        if t != '_PCA_3':
            df['feature'] = df.apply(lambda row: row.feature_asym.replace('_i','').replace('_j',''), axis=1)
            df = df.groupby(['feature']).agg({'importance_mean': ['mean'], 'importance_std': ['mean']}).reset_index()
        if rankings is None:
            rankings = pd.DataFrame(df['importance_mean'].values,index=df['feature'],columns=[names[i]])
        else:
            rankings[names[i]+t] = df['importance_mean'].values
    
    rankings['avg_importance'] = rankings.values.mean(axis=1)
    rankings = rankings.drop(index=['date','paird'])
    rankings = rankings.sort_values(by='avg_importance', axis=0,ascending=False)
    rankings.columns = ['Linear*','KNN*','G-Boost*','MLP*','avg_importance']

    rankings[rankings.columns] = MinMaxScaler().fit_transform(rankings[rankings.columns])
    rankings['avg_importance'] = rankings.values.mean(axis=1)
    rankings = rankings.rename(index={"new_cases_per_million":"New Covid cases",
    "new_deaths_per_million":"New Covid deaths",
    "c1_schoolclosing":"C1 School closing",
    "c2_workplaceclosing":"C2 Workplace closing",
    "c3_cancel_events":"C3 Cancel public events",
    "c4_restr_gather":"C4 Restrictions gatherings",
    "c5_closepublictransport":"C5 Close public transport",
    "c6_stay_home":"C6 Stay home requirements",
    "c7_restrnternal_move":"C7 Restr. Internal movement",
    "c8nt_trvl_controls":"C8 Inter. travel controls",
    "h2_testingpolicy":"H2 Testing policy",
    "h3_contacttracing":"H3 Contact tracing"})
    rankings = rankings.sort_values(by='avg_importance', axis=0,ascending=False)

    styler = rankings.style
    styler.set_properties(subset=rankings.columns, **{'width': '80px'})
    styler.set_precision(2)

    styler.background_gradient(cmap='plasma',vmin=rankings.min().min(),vmax=rankings.max().max())
    styler.format("{:.2%}")
    dfi.export(styler,file_path,fontsize=40)
    display(styler)

create_ranking_table('',"images/Feature_importance.pdf")

create_ranking_table('_date',"images/Feature_importance_date.pdf")

create_ranking_table('_pair_id',"images/Feature_importance_pair_id.pdf")

create_ranking_table('_date_pair_id',"images/Feature_importance_date_pair_id.pdf")

create_ranking_table('_PCA_3',"images/Feature_importance_PCA.pdf")


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,7.51%,87.19%,100.00%,74.70%,73.88%
C7 Restr. Internal movement,5.80%,100.00%,17.26%,100.00%,63.16%
C3 Cancel public events,28.56%,78.55%,36.49%,71.08%,57.98%
C4 Restrictions gatherings,42.77%,40.78%,71.87%,57.90%,55.32%
H2 Testing policy,100.00%,5.35%,37.60%,31.86%,40.68%
C2 Workplace closing,27.03%,18.36%,29.42%,54.55%,32.26%
C8 Inter. travel controls,4.02%,5.30%,59.75%,53.82%,30.78%
H3 Contact tracing,12.05%,16.32%,22.51%,44.13%,22.81%
C6 Stay home requirements,5.09%,6.92%,24.84%,47.53%,20.05%


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,90.65%,89.12%,100.00%,81.61%,92.28%
C4 Restrictions gatherings,77.93%,66.59%,58.23%,60.02%,66.15%
C7 Restr. Internal movement,17.26%,100.00%,18.38%,100.00%,65.21%
H2 Testing policy,100.00%,60.92%,48.54%,31.03%,57.88%
C6 Stay home requirements,31.91%,69.00%,22.78%,62.59%,49.14%
C8 Inter. travel controls,22.50%,47.96%,53.67%,58.31%,47.62%
C2 Workplace closing,41.92%,70.14%,14.12%,55.97%,47.36%
C3 Cancel public events,2.79%,74.23%,37.52%,56.90%,46.52%
New Covid deaths,91.31%,13.82%,52.61%,14.34%,38.58%


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C3 Cancel public events,70.31%,100.00%,51.65%,85.26%,80.27%
C1 School closing,10.25%,93.85%,80.87%,100.00%,76.99%
C4 Restrictions gatherings,50.92%,68.36%,59.45%,65.61%,64.36%
C7 Restr. Internal movement,13.12%,83.48%,12.79%,87.47%,50.86%
New Covid deaths,0.42%,70.10%,100.00%,0.00%,47.94%
New Covid cases,0.00%,62.40%,61.20%,24.36%,40.31%
H2 Testing policy,100.00%,2.18%,22.62%,9.13%,32.35%
C8 Inter. travel controls,19.64%,14.49%,47.52%,26.05%,28.41%
C6 Stay home requirements,3.95%,22.52%,14.12%,60.24%,25.56%


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,11.79%,72.75%,95.95%,84.36%,72.97%
C3 Cancel public events,3.99%,100.00%,45.58%,100.00%,64.68%
C4 Restrictions gatherings,53.39%,57.88%,67.44%,63.91%,64.28%
C6 Stay home requirements,100.00%,22.50%,14.91%,72.41%,51.08%
C7 Restr. Internal movement,16.95%,88.96%,9.67%,92.06%,50.73%
New Covid deaths,6.57%,10.05%,100.00%,0.00%,35.51%
C8 Inter. travel controls,7.00%,20.59%,42.69%,46.39%,30.75%
H2 Testing policy,14.24%,15.06%,26.77%,65.51%,30.69%
H3 Contact tracing,2.35%,28.29%,13.01%,22.60%,15.15%


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
Component_0,100.00%,100.00%,100.00%,100.00%,100.00%
Component_1,57.75%,56.74%,25.64%,19.66%,39.75%
Component_2,14.85%,46.00%,7.82%,2.52%,18.49%
New Covid deaths,4.03%,16.85%,10.56%,30.08%,15.73%
New Covid cases,0.00%,0.00%,0.00%,0.00%,0.00%
